# Showcase

This paper uses more complicated architecture for solving ***Flexible** Job-Shop Scheduling Problem*.  This paper is a bit more technical and complicate, so I didn't dive into the details. I just tried to make the model work. There were 3 checkpoints in the official repository, but I managed to run only one of them.

## Preprocessing

Since this model works on **FJSP**, I need to trasform our JSSP benchmarks to FJSP as in the model [fjsp-drl](/models/fjsp-drl/repo/Showcase%20fjsp-drl.ipynb). I will just reuse the same code.

In [1]:
import os
import argparse
import numpy as np

def get_all_instances_in_taillard_specification():
    '''Lists all instances in Taillard specification'''
    matching_files = []
    root_dir = "../../../../benchmarks/jssp"
    target_string = "Taillard_specification"

    for foldername, subfolders, filenames in os.walk(root_dir):
        for filename in filenames:
            filepath = os.path.join(foldername, filename)
            if target_string in filepath:
                matching_files.append(filepath)

    return matching_files

def parse_instance_taillard(filename):
    '''Parses instance written in Taillard specification: http://jobshop.jjvh.nl/explanation.php
    
      Args:
        filename - file containing the instance in Taillard specification

      Returns:
        number of jobs,
        number of machines,
        the processor times for each operation,
        the order for visiting the machines
    '''

    with open(filename, 'r') as f:
        # parse number of jobs J and machines M
        J, M = map(int, f.readline().split())

        # Initialize two empty numpy arrays with dimensions J x M
        processor_times = np.empty((J, M), dtype=int)
        orders_of_machines = np.empty((J, M), dtype=int)
    
        # Read the next J lines containing processor times
        for i in range(J):
            processor_times[i] = list(map(int, f.readline().split()))
    
        # Read the next J lines containing orders of machines
        for i in range(J):
            orders_of_machines[i] = list(map(int, f.readline().split()))

        return J, M, processor_times, orders_of_machines

def jssp_taillard_to_fjsp(filename: str) -> str:
    '''Transforms JSSP instance in Taillard's specification to FJSP instance
       and stores it in a temporary file
    
      Args:
        filename - name of the file with JSSP instance in Taillard's specification
        
      Returns:
        string - filename of the equivalent FJSP instance 
    '''
    # parse JSSP Taillard instance
    J, M, processor_times, orders_of_machines = parse_instance_taillard(filename)
    
    # convert JSSP to FJSP
    with open("/tmp/fjsp_" + filename.split("/")[-1], 'w') as f:
        # write number of jobs, number of machines, and jobs/machines (which is always 1 for JSSP)
        f.write(str(J) + "   " + str(M) + "   1\n")
        
        # each line is a job
        for i in range(J):
            # each line starts with the number of operations in a job
            number_of_operations = len(processor_times[i])
            f.write(str(number_of_operations) + "  ")
            
            # print the operation as a tuple (number of available machines, current machine, processing time)
            for j in range(number_of_operations):
                f.write(" 1   " + str(orders_of_machines[i][j]) + "   " + str(processor_times[i][j]) + "  ")
                
            f.write('\n')
            
    return "/tmp/fjsp_" + filename.split("/")[-1]

The original code is a bit messy, so I had to make a small wrapper around the original code to make it nicer.

In [283]:
import time
from Params import configs
from validation_realWorld import test
from torch.utils.data import DataLoader
from PPOwithValue import PPO
from DataRead import getdata
import torch
import os
import copy
import itertools
from FJSP_Env import FJSP
from mb_agg import g_pool_cal
from mb_agg import *
from copy import deepcopy

def validate(vali_set,batch_size, policy_job,policy_mch,num_operation,number_of_task,Data, plan: list | None = None):
    policy_job = copy.deepcopy(policy_job)
    policy_mch = copy.deepcopy(policy_mch)
    policy_job.eval()
    policy_mch.eval()
    def eval_model_bat(bat, plan: list | None = None):
        actions = []
        start_times = []
        with torch.no_grad():
            data = bat.numpy()

            env = FJSP(n_j=Data['n'], n_m=configs.n_m,EachJob_num_operation=num_operation)
            device = torch.device(configs.device)
            g_pool_step = g_pool_cal(graph_pool_type=configs.graph_pool_type,
                                     batch_size=torch.Size(
                                         [batch_size, number_of_task, number_of_task]),
                                     n_nodes=number_of_task,
                                     device=device)

            adj, fea, candidate, mask, mask_mch, dur, mch_time, job_time = env.reset(data)
            first_task = []
            pretask = []

            ep_rewards = - env.initQuality
            rewards = []
            env_mask_mch = torch.from_numpy(np.copy(mask_mch)).to(device)
            env_dur = torch.from_numpy(np.copy(dur)).float().to(device)
            pool=None
            for j in itertools.count():
                env_adj = aggr_obs(deepcopy(adj).to(device).to_sparse(), number_of_task)

                env_fea = torch.from_numpy(np.copy(fea)).float().to(device)
                env_fea = deepcopy(env_fea).reshape(-1, env_fea.size(-1))
                env_candidate = torch.from_numpy(np.copy(candidate)).long().to(device)
                env_mask = torch.from_numpy(np.copy(mask)).to(device)
                env_mch_time = torch.from_numpy(np.copy(mch_time)).float().to(device)
                
                if plan is not None and j < len(plan):
                    # choose action from partial plan if possible
                    action, a_idx, log_a, action_node, _, mask_mch_action, hx = plan[j]
                else:
                    # if no action was chosen from partial plan, use agent to choose
                    action, a_idx, log_a, action_node, _, mask_mch_action, hx = policy_job(x=env_fea,
                                                                                                   graph_pool=g_pool_step,
                                                                                                   padded_nei=None,
                                                                                                   adj=env_adj,
                                                                                                   candidate=env_candidate
                                                                                                   , mask=env_mask
    
                                                                                                   , mask_mch=env_mask_mch
                                                                                                   , dur=env_dur
                                                                                                   , a_index=0
                                                                                                   , old_action=0
                                                                                           , mch_pool=pool
                                                                                                   ,old_policy=True,
                                                                                                    T=1
                                                                                                   ,greedy=True
                                                                                                   )
                actions.append((action, a_idx, log_a, action_node, _, mask_mch_action, hx))
                pi_mch,_,pool = policy_mch(action_node, hx, mask_mch_action, env_mch_time)
                _, mch_a = pi_mch.squeeze(-1).max(1)

                if j == 0:
                    first_task = action.type(torch.long).to(device)
                pretask = action.type(torch.long).to(device)

                # make an action
                adj, fea, reward, done, candidate, mask,job,_,mch_time,job_time = env.step(action.cpu().numpy(), mch_a)

                action = action.cpu().numpy()
                row = np.where(action[0] <= env.last_col[0])[0][0]
                col = action[0] - env.first_col[0][row]
                start_times.append(env.temp1[0][row][col] - env.dur_a)

                if env.done():
                    break

            cost = env.mchsEndTimes.max(-1).max(-1)
            # assert cost[0] == cost[1], f/"First cost and second cost are not equal {cost[0], cost[1]}"
        return cost[0], actions, start_times

    for bat in vali_set:
        assert torch.equal(bat[0], bat[1]), "First and second matrix in batch are not equal"
        return eval_model_bat(bat, plan)

def test(filepath, datafile, plan: list | None = None):

    Data = getdata(datafile)

    n_j = Data['n']
    n_m = Data['m']

    ppo = PPO(configs.lr, configs.gamma, configs.k_epochs, configs.eps_clip,
                n_j=n_j,
                n_m=n_m,
                num_layers=configs.num_layers,
                neighbor_pooling_type=configs.neighbor_pooling_type,
                input_dim=configs.input_dim,
                hidden_dim=configs.hidden_dim,
                num_mlp_layers_feature_extract=configs.num_mlp_layers_feature_extract,
                num_mlp_layers_actor=configs.num_mlp_layers_actor,
                hidden_dim_actor=configs.hidden_dim_actor,
                num_mlp_layers_critic=configs.num_mlp_layers_critic,
                hidden_dim_critic=configs.hidden_dim_critic)

    job_path = './{}.pth'.format('policy_job')
    mch_path = './{}.pth'.format('policy_mch')

    job_path = os.path.join(filepath,job_path)
    mch_path = os.path.join(filepath, mch_path)

    ppo.policy_job.load_state_dict(torch.load(job_path, map_location=torch.device('cpu')))
    ppo.policy_mch.load_state_dict(torch.load(mch_path, map_location=torch.device('cpu')))

    batch_size = 2
    num_operations = []
    num_operation = []
    for i in Data['J']:
        num_operation.append(Data['OJ'][i][-1])
    num_operation_max = np.array(num_operation).max()

    time_window = np.zeros(shape=(Data['n'], num_operation_max, Data['m']))

    data_set = []
    for i in range(Data['n']):
        for j in Data['OJ'][i+1]:
            mchForJob = Data['operations_machines'][(i + 1, j)]
            for k in mchForJob:
                time_window[i][j-1][k - 1] = Data['operations_times'][(i + 1, j, k)]


    for i in range(batch_size):
        num_operations.append(num_operation)
        data_set.append(time_window)
    data_set = np.array(data_set)

    num_operation = np.array(num_operations)
    number_of_tasks = num_operation.sum(axis=1)[0]
    number_of_tasks = int(number_of_tasks)

    valid_loader = DataLoader(data_set, batch_size=batch_size)
    makespan, actions, start_times = validate(valid_loader,batch_size, ppo.policy_job, ppo.policy_mch,num_operation,number_of_tasks,Data, plan)

    return makespan, actions, start_times

def solve_fjsp_instance(model, instance, plan: list | None = None):
    '''Solves FJSP instance using given model

    Args:
      model - model to use for solving the instance
      instance - instance to be solved
      plan - preset actions to make by agent

    Returns:
      makespan of the instance,
      actions made by agent,
      time it took to solve the instance
    '''
    # get number of machines
    with open(instance, 'r') as f:
        M = int(f.readline().strip().split()[1])

    # override the value of number of machines in the configs according to the instance 
    # this is not handled in the original code and causes errors
    setattr(configs, 'n_m', M)

    # solve the instance
    start = time.time()
    makespan, actions, start_times = test(model, instance, plan)
    end = time.time()

    return makespan, actions, start_times, end - start

# makespan, actions, start_times, run_time = solve_fjsp_instance("saved_network/FJSP_J15M15/best_value0", "/Users/marosbratko/Graph-neural-networks-and-deep-reinforcement-learning-in-job-scheduling/benchmarks/fjsp/0_BehnkeGeiger/Behnke1.fjs")
# actions = actions[:len(actions) // 2]
# makespan, actions, start_times, run_time = solve_fjsp_instance("saved_network/FJSP_J15M15/best_value0", "/Users/marosbratko/Graph-neural-networks-and-deep-reinforcement-learning-in-job-scheduling/benchmarks/fjsp/0_BehnkeGeiger/Behnke1.fjs", plan=actions)
# print(makespan, run_time)

## Run the model on instances from the paper available in the official repository

Only subset of instances from the paper are available in the official repository.

In [3]:
from validation_realWorld import get_imlist

MODEL_PATH = 'saved_network/FJSP_J15M15/best_value0'
INSTANCES_PATH = './FJSSPinstances/M15'

for instance in sorted(get_imlist(INSTANCES_PATH)):
    makespan, run_time = solve_fjsp_instance(MODEL_PATH, instance)
    print(f"Instance: {instance.split('/')[-1]}, makespan: {makespan}, time: {np.round(run_time, 2)}")

/Users/marosbratko/Graph-neural-networks-and-deep-reinforcement-learning-in-job-scheduling/models/End-to-end-DRL-for-FJSP/repo/FJSP_RealWorld/mb_agg.py:44: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:607.)
  graph_pool = torch.sparse.FloatTensor(idx, elem,
/Users/marosbratko/Graph-neural-networks-and-deep-reinforcement-learning-in-job-scheduling/models/End-to-end-DRL-for-FJSP/repo/FJSP_RealWorld/FJSP_Env.py:221: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:264.)
  self.adj = torch.tensor(self.adj)


Instance: HurinkVdata39.fjs, makespan: 972.0, time: 1.63
Instance: HurinkVdata40.fjs, makespan: 1030.0, time: 0.54
Instance: HurinkVdata41.fjs, makespan: 960.0, time: 0.55
Instance: HurinkVdata42.fjs, makespan: 988.0, time: 0.64
Instance: HurinkVdata43.fjs, makespan: 974.0, time: 0.66
Instance: HurinkVdata46.fjs, makespan: 524.0, time: 0.84
Instance: HurinkVdata47.fjs, makespan: 560.5, time: 0.96
Instance: HurinkVdata48.fjs, makespan: 555.0, time: 0.9


## Run the model on JSSP benchmarks

In [4]:
MODEL_PATH = 'saved_network/FJSP_J15M15/best_value0'

exceptions = [
    '../../../../benchmarks/jssp/orb_instances/Taillard_specification/orb07.txt' # this instance gets stuck while solving, I haven't figured out why
]

for instance in sorted(get_all_instances_in_taillard_specification()):
    if instance in exceptions:
        print('Skipping', instance)
        continue

    makespan, run_time = solve_fjsp_instance(MODEL_PATH, jssp_taillard_to_fjsp(instance))
    print(f"Instance: {instance.split('/')[-1]}, makespan: {makespan}, time: {np.round(run_time, 2)}")

Instance: abz5.txt, makespan: 1382.0, time: 0.22
Instance: abz6.txt, makespan: 1194.0, time: 0.2
Instance: abz7.txt, makespan: 780.0, time: 0.88
Instance: abz8.txt, makespan: 866.5, time: 0.89
Instance: abz9.txt, makespan: 913.0, time: 0.85
Instance: dmu01.txt, makespan: 3391.0, time: 0.83
Instance: dmu02.txt, makespan: 3561.0, time: 0.9
Instance: dmu03.txt, makespan: 3427.0, time: 0.88
Instance: dmu04.txt, makespan: 3395.0, time: 0.86
Instance: dmu05.txt, makespan: 3490.0, time: 0.86
Instance: dmu06.txt, makespan: 4276.0, time: 1.34
Instance: dmu07.txt, makespan: 3914.0, time: 1.27
Instance: dmu08.txt, makespan: 3879.0, time: 1.26
Instance: dmu09.txt, makespan: 4250.0, time: 1.31
Instance: dmu10.txt, makespan: 3859.0, time: 1.35
Instance: dmu11.txt, makespan: 4381.0, time: 1.61
Instance: dmu12.txt, makespan: 4587.0, time: 1.67
Instance: dmu13.txt, makespan: 4995.0, time: 1.66
Instance: dmu14.txt, makespan: 4212.0, time: 1.63
Instance: dmu15.txt, makespan: 4222.0, time: 1.65
Instance: 

# Run the model on FJSP benchmarks

In [5]:
def get_all_fjsp_instances():
    '''Lists all FJSP instances'''
    matching_files = []
    root_dir = "../../../../benchmarks/fjsp"
    target_string = ".fjs"

    for foldername, subfolders, filenames in os.walk(root_dir):
        for filename in filenames:
            filepath = os.path.join(foldername, filename)
            if target_string in filepath:
                matching_files.append(filepath)

    return matching_files

In [6]:
MODEL_PATH = 'saved_network/FJSP_J15M15/best_value0'

for instance in sorted(get_all_fjsp_instances()):
    try:
        makespan, run_time = solve_fjsp_instance(MODEL_PATH, instance)
        print(f"Instance: {instance.split('/')[-1]}, makespan: {makespan}, time: {np.round(run_time, 2)}")
    except Exception as e:
        print(f"Failed to solve instance {instance}, error: {e}")

Instance: Behnke1.fjs, makespan: 110.0, time: 0.19
Instance: Behnke10.fjs, makespan: 163.5, time: 0.35
Instance: Behnke11.fjs, makespan: 286.0, time: 1.28
Instance: Behnke12.fjs, makespan: 276.0, time: 1.23
Instance: Behnke13.fjs, makespan: 301.0, time: 1.2
Instance: Behnke14.fjs, makespan: 300.5, time: 1.17
Instance: Behnke15.fjs, makespan: 286.0, time: 1.2
Instance: Behnke16.fjs, makespan: 508.5, time: 5.49
Instance: Behnke17.fjs, makespan: 486.5, time: 5.31
Instance: Behnke18.fjs, makespan: 501.5, time: 5.54
Instance: Behnke19.fjs, makespan: 495.0, time: 5.64
Instance: Behnke2.fjs, makespan: 110.0, time: 0.14
Instance: Behnke20.fjs, makespan: 491.5, time: 5.72
Instance: Behnke21.fjs, makespan: 105.0, time: 0.16
Instance: Behnke22.fjs, makespan: 107.0, time: 0.15
Instance: Behnke23.fjs, makespan: 106.0, time: 0.16
Instance: Behnke24.fjs, makespan: 108.0, time: 0.18
Instance: Behnke25.fjs, makespan: 96.0, time: 0.18
Instance: Behnke26.fjs, makespan: 150.0, time: 0.4
Instance: Behnke27

/Users/marosbratko/Graph-neural-networks-and-deep-reinforcement-learning-in-job-scheduling/models/End-to-end-DRL-for-FJSP/repo/FJSP_RealWorld/FJSP_Env.py:262: RuntimeWarning: Mean of empty slice.
  self.dur[t][i][j] = [durmch.mean() if i <= 0 else i for i in self.dur[t][i][j]]
/Users/marosbratko/Library/Caches/pypoetry/virtualenvs/end-to-end-drl-for-fjsp-cBOGAF6U-py3.10/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)


Instance: BrandimarteMk10.fjs, makespan: 248.5, time: 0.75
Instance: BrandimarteMk11.fjs, makespan: 673.5, time: 0.53
Instance: BrandimarteMk12.fjs, makespan: 566.0, time: 0.64
Instance: BrandimarteMk13.fjs, makespan: 484.0, time: 0.76
Instance: BrandimarteMk14.fjs, makespan: 768.0, time: 1.11
Instance: BrandimarteMk15.fjs, makespan: 435.0, time: 1.07
Instance: BrandimarteMk2.fjs, makespan: 34.0, time: 0.17
Instance: BrandimarteMk3.fjs, makespan: 207.0, time: 0.48
Instance: BrandimarteMk4.fjs, makespan: 74.0, time: 0.28
Instance: BrandimarteMk5.fjs, makespan: 177.0, time: 0.29
Instance: BrandimarteMk6.fjs, makespan: 84.0, time: 0.44
Instance: BrandimarteMk7.fjs, makespan: 153.0, time: 0.27
Instance: BrandimarteMk8.fjs, makespan: 531.0, time: 0.75
Instance: BrandimarteMk9.fjs, makespan: 337.0, time: 0.73
Instance: HurinkSdata1.fjs, makespan: 64.0, time: 0.09
Instance: HurinkSdata10.fjs, makespan: 1088.0, time: 0.21
Instance: HurinkSdata11.fjs, makespan: 1031.0, time: 0.21
Instance: Huri

# Dynamic FJSP

In dynamic FJSP, only a subset of jobs is known at the beginning. The rest of jobs arrives dynamically online.

The following attempt to expand this model to being dynamic is inspired by paper [Large-scale Dynamic Scheduling for Flexible Job-shop with Random Arrivals of New Jobs by Hierarchical Reinforcement Learning](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=10114974&tag=1), where authors schedule newly incoming jobs and reschedule not yet executed operations, already executed operations can not be rescheduled. During each rescheduling, they formulate static FJSP and solve it. They use cache for incoming jobs and an agent choosing either to add jobs from cache to scheduling problem, or keep them in cache. I will skip this agent and always add new jobs to scheduling problem.

Similarly to the paper, I will model the arrival of new jobs as poisson process with average arrival time following an exponential distribution.

In [235]:
from datetime import datetime

def get_dynamic_fjsp(instance):
    '''Turns static FJSP instance to dynamic

      Args:
        filename of static FJSP instance

      Returns:
        list of jobs known at the beginning
        dictionary of arriving jobs as  as {time_of_arrival: (operations, machines)} 
    '''
    # get number of jobs and average time per operation from 
    total_time_of_operations = 0
    number_of_operations = 0
    number_of_operations_per_job = []
    jobs = []
    with open(instance, 'r') as f:
        J, M = list(map(lambda o: int(o), f.readline().removesuffix('\n').split()[:2]))

        for line in f:
            jobs.append(line.removesuffix('\n'))
            number_of_operations_per_job.append(int(line[0]))
            line = line.split()[1:]

            while line:
                # get number of available machines for operation
                number_of_machines_for_current_operation = int(line[0])
                number_of_operations += number_of_machines_for_current_operation

                # get possible durations of operation
                current_operation = list(map(lambda arg: int(arg), line[1:1+number_of_machines_for_current_operation*2]))
                total_time_of_operations += sum(current_operation[1::2])

                # move to next operation
                line = line[1+number_of_machines_for_current_operation*2:]

    # calculate average time between arrivals (beta=1/lambda)
    average_operation_duration = total_time_of_operations // number_of_operations
    average_time_between_arrivals = average_operation_duration * sum(number_of_operations_per_job) / len(number_of_operations_per_job) / M

    # shuffle jobs
    indices = np.arange(J)
    np.random.shuffle(indices)

    # # separate jobs into known jobs and arriving jobs
    jobs_known_at_the_beginning = [jobs[index] for index in indices[J//2:]]
    arriving_jobs_indeces = indices[:J//2]

    # create arriving jobs
    t = 0
    arriving_jobs = {}
    for index in arriving_jobs_indeces:
        t += int(np.random.exponential(scale=average_time_between_arrivals))
        arriving_jobs[t] = jobs[index]

    return jobs_known_at_the_beginning, arriving_jobs, M

def save_static_fjsp(jobs, number_of_machines):
    '''Saves list of jobs as static FJSP instance
        
      Args:
        list of jobs to save
        number of machines in instance

      Returns:
        filename where JSSP instance was saved to
    '''
    J, M = len(jobs), number_of_machines
    formatted_datetime = datetime.now().strftime("%Y%m%d%H%M%S")
    with open(f"/tmp/{J}_{M}_{formatted_datetime}.txt", 'w') as f:
        f.write(f"{J} {M} 0\n")
        for job in jobs:
            f.write(job + '\n') 

    return f"/tmp/{J}_{M}_{formatted_datetime}.txt"

# jobs_known_at_the_beginning, arriving_jobs, number_of_machines = get_dynamic_fjsp('/Users/marosbratko/Graph-neural-networks-and-deep-reinforcement-learning-in-job-scheduling/benchmarks/fjsp/0_BehnkeGeiger/Behnke1.fjs')
# instance = save_static_fjsp(jobs_known_at_the_beginning, number_of_machines)
# print(instance)
# a, b, c, d = solve_fjsp_instance("saved_network/FJSP_J15M15/best_value0", instance)
# print(a)

In [285]:
def solve_dynamic_fjsp(model, instance):
    '''Turns static FJSP instance to dynamic and solves it

      Args: 
        model to use
        instance to solve

      Returns: 
        makespan, run
    '''
    # turn static JSSP instance to dynamic
    known_jobs, arriving_jobs, number_of_machines = get_dynamic_fjsp(instance)
    print(f"instance={instance.split('/')[-1]}, known_jobs={len(known_jobs)}, arriving_jobs={len(arriving_jobs)}")
    latest_time_of_arrival = max(arriving_jobs)
    print(f"Latest job arrives at {latest_time_of_arrival}")
    
    # solve static JSSP with jobs known initially
    makespan, actions, start_times, _ = solve_fjsp_instance(model, save_static_fjsp(known_jobs, number_of_machines))
    t = 0
    while True:
        t += 1
        
        # no jobs left
        if not arriving_jobs:
            break
    
        # no job arrived
        if not t in arriving_jobs:
            continue
    
        # new job arrived, remove not yet executed operations from the plan
        partial_plan = []
        flag = False
        for action, start_time in zip(reversed(actions), reversed(start_times)):
            if start_time < t:
                flag = True

            if flag:
                partial_plan.append(action)

        partial_plan = list(reversed(partial_plan))
        # add new job to the plan, with times shifted to current time t
        new_job = arriving_jobs.pop(t)
        known_jobs.append(new_job)
        
        # create new schedule WHILE REUSING THE ALREADY EXECUTED PLAN
        makespan, actions, start_times, _ = solve_fjsp_instance(model, save_static_fjsp(known_jobs, number_of_machines), plan=partial_plan)
    return makespan

# Dynamic experiment

In [ ]:
MODEL_PATH = 'saved_network/FJSP_J15M15/best_value0'

for instance in sorted(get_all_fjsp_instances())[:10]:
    try:
        makespan = solve_dynamic_fjsp(MODEL_PATH, instance)
        print(f"Instance: {instance.split('/')[-1]}, makespan: {makespan}, time: {np.round(run_time, 2)}")
    except Exception as e:
        print(f"Failed to solve instance {instance}, error: {e}")

instance=Behnke1.fjs, known_jobs=5, arriving_jobs=4
Latest job arrives at 13
Instance: Behnke1.fjs, makespan: 108.0, time: 5.04
instance=Behnke10.fjs, known_jobs=10, arriving_jobs=9
Latest job arrives at 67
Instance: Behnke10.fjs, makespan: 156.0, time: 5.04
instance=Behnke11.fjs, known_jobs=25, arriving_jobs=18
Latest job arrives at 81
Instance: Behnke11.fjs, makespan: 260.0, time: 5.04
instance=Behnke12.fjs, known_jobs=25, arriving_jobs=19
Latest job arrives at 123
Instance: Behnke12.fjs, makespan: 252.0, time: 5.04
instance=Behnke13.fjs, known_jobs=25, arriving_jobs=23
Latest job arrives at 116
Instance: Behnke13.fjs, makespan: 275.0, time: 5.04
instance=Behnke14.fjs, known_jobs=25, arriving_jobs=20
Latest job arrives at 105
Instance: Behnke14.fjs, makespan: 268.0, time: 5.04
instance=Behnke15.fjs, known_jobs=25, arriving_jobs=21
Latest job arrives at 70
Instance: Behnke15.fjs, makespan: 244.0, time: 5.04
instance=Behnke16.fjs, known_jobs=50, arriving_jobs=43
Latest job arrives at 2